# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

%matplotlib inline

model_name = './models/Conv-3-1-Aug-mean-Drop-1e-4'
# Load signnames.csv that contains id -> name mappings for each id.
all_labels = pd.read_csv('./signnames.csv').values;
num_labels = len(all_labels)

In [ ]:
# Load pickled data

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'data/train.p'
validation_file= 'data/valid.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = len(y_train)

# TODO: Number of validation examples
n_validation = len(y_valid)

# TODO: Number of testing examples.
n_test = len(y_test)

# TODO: What's the shape of an traffic sign image?
image_shape = np.array(X_train[0]).shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = np.unique(y_train).size

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.

# Visualizations will be shown in the notebook.

signIndxs = []
for indx in range(n_classes):
    signIndx = np.random.choice([i for i in range(len(y_train)) if y_train[i] == indx],5)
    signIndxs.append(signIndx)

In [ ]:
print('Traffic Signs Samples')
for indx in range(n_classes):
    class_imgs = signIndxs[indx]
    class_label = all_labels[indx]
    print('{}. {}.'.format(class_label[0], class_label[1]))
    fig = plt.figure(indx, figsize=(5,1))
    for img in range(len(class_imgs)):
        ax0 = fig.add_subplot(1, 5, img+1)
        ax0.imshow(X_train[class_imgs[img]])
        ax0.axis('off')
    
    plt.show()

In [ ]:
plt.hist(y_train, bins=n_classes)
plt.title('Train Distribution')
plt.show()

In [ ]:
plt.hist(y_valid, bins=n_classes)
plt.title('Validation Distribution')
plt.show()

In [ ]:
plt.hist(y_test, bins=n_classes)
plt.title('Test Distribution')
plt.show()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
import cv2

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
def gray_clahe(x):
    h, w, c = x.shape
    x_gray = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
    x_clahe = clahe.apply(x_gray)
    return x_clahe.reshape(h, w, 1)

In [ ]:
from scipy.ndimage.interpolation import shift

def shift_img(x, shift_x):
    return shift(x, shift_x)

In [ ]:
from scipy.ndimage.interpolation import rotate

def rotate_img(x, angle_x):
    return rotate(x, angle_x, reshape=False)

In [ ]:
from scipy.ndimage.interpolation import zoom

def scale_img(x, zoom_img):
    scaled = zoom(x, zoom_img)
    if zoom_img < 1.0:
        pad = (x.shape[0] - scaled.shape[0]) // 2
        scaled = np.pad(scaled, ((pad,pad),(pad,pad), (0,0)), 'constant')
    else:
        crop = (scaled.shape[0] - x.shape[0]) // 2
        scaled = scaled[crop:scaled.shape[0] - crop, crop:scaled.shape[0] - crop,:]
    return scaled

In [ ]:
def augment(X, y):
    X_news = []
    y_news = []
    imres = 32
    count = 0
    hist, _ = np.histogram(y, bins=range(np.unique(y).size + 1))
    par = True
    a1 = 1
    for indx in range(len(X)):
        x_origin, y_origin = X[indx], y[indx]
        x_origin = gray_clahe(x_origin)
        X_news.append(x_origin)
        y_news.append(y_origin)
        imgs_count = hist[y_origin]
        
        if imgs_count <= 240:
            X_news.append(shift_img(x_origin, (2,2,0)))
            y_news.append(y_origin)
            X_news.append(shift_img(x_origin, (-2,2,0)))
            y_news.append(y_origin)
            X_news.append(shift_img(x_origin, (2,-2,0)))
            y_news.append(y_origin)
            X_news.append(shift_img(x_origin, (-2,-2,0)))
            y_news.append(y_origin)
            
            X_news.append(rotate_img(x_origin, 10))
            y_news.append(y_origin)
            X_news.append(rotate_img(x_origin, 5))
            y_news.append(y_origin)
            X_news.append(rotate_img(x_origin, -10))
            y_news.append(y_origin)
            X_news.append(rotate_img(x_origin, -5))
            y_news.append(y_origin)
            
            X_news.append(scale_img(x_origin, 0.8125))
            y_news.append(y_origin)
            X_news.append(scale_img(x_origin, 1.1875))
            y_news.append(y_origin)
            
            X_news.append(x_origin -50)
            y_news.append(y_origin)
            
            X_news.append(x_origin - 75)
            y_news.append(y_origin)
            

        elif imgs_count > 240 and imgs_count <= 450:
            X_news.append(shift_img(x_origin, (2,2,0)))
            y_news.append(y_origin)
            X_news.append(shift_img(x_origin, (-2,2,0)))
            y_news.append(y_origin)
            X_news.append(shift_img(x_origin, (2,-2,0)))
            y_news.append(y_origin)
            
            X_news.append(rotate_img(x_origin, 10))
            y_news.append(y_origin)
            X_news.append(rotate_img(x_origin, 5))
            y_news.append(y_origin)
            
            X_news.append(scale_img(x_origin, 0.8125))
            y_news.append(y_origin)
            
            X_news.append(x_origin + (a1 * 25))
            y_news.append(y_origin)
            a1 *= -1 

        elif imgs_count > 450 and imgs_count <= 800:
            X_news.append(shift_img(x_origin, (-2,-2,0)))
            y_news.append(y_origin)
            
            X_news.append(rotate_img(x_origin, -10))
            y_news.append(y_origin)
            
            X_news.append(scale_img(x_origin, 1.1875))
            y_news.append(y_origin)            

        elif imgs_count > 800 and imgs_count <= 990:
            X_news.append(shift_img(x_origin, (-2,-2,0)))
            y_news.append(y_origin)
            
            X_news.append(rotate_img(x_origin, -5))
            y_news.append(y_origin)
        elif imgs_count > 990 and imgs_count <= 1320:
            X_news.append(scale_img(x_origin, 0.8125))
            y_news.append(y_origin)
        else:
            if par:
                X_news.append(scale_img(x_origin, 1.1875))
                y_news.append(y_origin)
            par = not par


        count +=1

    return np.array(X_news).reshape(len(X_news), imres, imres, 1), np.array(y_news)

In [ ]:
%%time
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

# train gray clahe augmented
X_augmented, y_train_norm = augment(X_train, y_train)
X_augmented = np.array(X_augmented).reshape(len(X_augmented), 32, 32, 1)


# valid gray clahe
X_valid_gray_clahe = [gray_clahe(x) for x in X_valid]
X_valid_gray_clahe = np.array(X_valid_gray_clahe).reshape(len(X_valid_gray_clahe), 32, 32, 1)

# Normalization mean-std
X_train_norm = np.array(X_augmented, dtype=np.float32)
X_valid_norm = np.array(X_valid_gray_clahe, dtype=np.float32)
train_mean = np.mean(X_train_norm)
train_std = np.std(X_train_norm, axis=0) 

In [ ]:
plt.hist(y_train_norm, bins=n_classes)
plt.title('Augmented Train Distribution')
plt.show()

In [ ]:
from sklearn.utils import shuffle
X_train_norm, y_train_norm = shuffle(X_train_norm, y_train_norm)

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
EPOCHS = 25
BATCH_SIZE = 128

In [ ]:
def convLayer(x, filter_conv, strides_conv, keepprob, ksize_pool=None, strides_pool=None, padding_conv='SAME', mu=0, sigma=0.1):
    
    # Convolutional
    conv_W = tf.Variable(tf.truncated_normal(shape=filter_conv, mean = mu, stddev = sigma))
    conv_b = tf.Variable(tf.zeros(filter_conv[3]))
    conv   = tf.nn.conv2d(x, conv_W, strides=strides_conv, padding=padding_conv, name="conv") + conv_b

    # Activation.
    conv = tf.nn.relu(conv)

    # Dropout
    conv = tf.cond(with_drop,
                  lambda: tf.nn.dropout(conv, keepprob),
                  lambda: conv)
    
    # Pooling
    if ksize_pool is not None:
        conv = tf.nn.max_pool(conv, ksize=ksize_pool, strides=strides_pool, padding='VALID')
        
    return conv

In [ ]:
from tensorflow.contrib.layers import flatten
import math

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    strides_conv = [1, 1, 1, 1]
    ksize_pool   = [1, 2, 2, 1]
    strides_pool  = [1, 2, 2, 1]
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 14x14x46.
    with tf.variable_scope("conv1"):
        conv1 = convLayer(x, (5, 5, 1, 46), strides_conv , 0.5, ksize_pool, strides_pool, padding_conv='VALID',
                          sigma=math.sqrt(2.0/(5.0*5.0*46.0)))
    
    # Layer 2: Convolutional. Input = 14x14x46. Output = 5x5x92.
    with tf.variable_scope("conv2"):
        conv2 = convLayer(conv1, (5, 5, 46, 92), strides_conv, 0.5, ksize_pool, strides_pool, padding_conv='VALID',
                          sigma=math.sqrt(2.0/(5.0*5.0*92.0))) 
    
    # Layer 3: Convolutional. Input = 5x5x92. Output = 4x4x184.
    with tf.variable_scope("conv3"):
        conv3 = convLayer(conv2, (3, 3, 92, 184), strides_conv, 0.5, ksize_pool, [1, 1, 1, 1], padding_conv='SAME',
                          sigma=math.sqrt(2.0/(5.0*5.0*184.0))) 
    
    # Flatten. Input = 4x4x184. Output = 2944.
    fc0   = flatten(conv3)
    
    neuron_fc1 = 4*4*184
    
    # Layer 4: Fully Connected. Input = 2944. Output = 1000.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(neuron_fc1, 1000), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(1000))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Layer 4: Activation.
    fc1    = tf.nn.relu(fc1)
    # Layer 4: Dropout.
    fc1 = tf.cond(with_drop,
                 lambda: tf.nn.dropout(fc1,0.5),
                 lambda: fc1)

    # Layer 5: Fully Connected. Input = 1000 Output = 43.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(1000, n_classes), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(n_classes))
    logits = tf.add(tf.matmul(fc1, fc2_W), fc2_b, name='logits')
    
    return logits

In [ ]:
tf.set_random_seed(542)
x = tf.placeholder(tf.float32, (None, 32, 32, 1), name='x')
y = tf.placeholder(tf.int32, (None), name='y')
with_drop = tf.placeholder(tf.bool, name='with_drop')
one_hot_y = tf.one_hot(y, n_classes)
mean = tf.Variable(train_mean)
std = tf.Variable(train_std)

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
rate = 0.0001

normalization = tf.div(tf.subtract(x, mean), std, name='normalization')
logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy_operation')

In [ ]:
def evaluate(X_data, y_data, drop=False):
    num_examples = len(X_data)
    total_accuracy = 0
    total_loss = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        loss, accuracy = sess.run([loss_operation, accuracy_operation], feed_dict={x: batch_x, y: batch_y, with_drop: drop})
        
        total_loss += (loss * len(batch_x))
        total_accuracy += (accuracy * len(batch_x))
        
    return total_loss / num_examples, total_accuracy / num_examples

In [ ]:
%%time
train_loss_history = []
valid_loss_history = []
train_accuracy_history = []
valid_accuracy_history = []

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train_norm)
    
    print("Training...")
    print()
    X_train_norm = sess.run(normalization, feed_dict={x: X_train_norm})
    X_valid_norm = sess.run(normalization, feed_dict={x: X_valid_norm})
    for i in range(EPOCHS):
        X_train_norm, y_train_norm = shuffle(X_train_norm, y_train_norm)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_norm[offset:end], y_train_norm[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, with_drop: True})
            
        train_loss, train_accuracy = evaluate(X_train_norm, y_train_norm)
        train_loss_history.append(train_loss)
        train_accuracy_history.append(train_accuracy)
        validation_loss, validation_accuracy = evaluate(X_valid_norm, y_valid)
        valid_loss_history.append(validation_loss)
        valid_accuracy_history.append(validation_accuracy)
        
        print("EPOCH {} ...".format(i+1))
        print("Train Accuracy = {:.3f} --- Validation Accuracy = {:.3f}".format(train_accuracy, validation_accuracy))
        print()
        
    saver.save(sess, model_name)
    print("Model saved")
    print("Train Loss = {:.3f} --- Validation Loss = {:.3f}".format(train_loss_history[EPOCHS-1], 
                                                                    valid_loss_history[EPOCHS-1]))
    
loss_plot = plt.subplot(2,1,1)
loss_plot.set_title('Loss')
loss_plot.plot(train_loss_history, 'r', label='Training Loss')
loss_plot.plot(valid_loss_history, 'b', label='Validation Loss')
loss_plot.set_xlim([0, EPOCHS])
loss_plot.set_ylim([0, 4.0])
loss_plot.legend(loc=1)
accuracy_plot = plt.subplot(2,1,2)
accuracy_plot.set_title('Accuracy')
accuracy_plot.plot(train_accuracy_history, 'r', label='Training Accuracy')
accuracy_plot.plot(valid_accuracy_history, 'b', label='Validation Accuracy')
accuracy_plot.set_xlim([0, EPOCHS])
accuracy_plot.set_ylim([0.5, 1.1])
accuracy_plot.legend(loc=4)
plt.tight_layout()
plt.show()

In [ ]:
X_test_gray_clahe = [gray_clahe(x) for x in X_test]
X_test_norm = np.array(X_test_gray_clahe, dtype=np.float32).reshape(len(X_test_gray_clahe), 32, 32, 1)
# X_test_norm = np.array(X_test, dtype=np.float32)
with tf.Session() as sess:

    saver = tf.train.import_meta_graph('{}.meta'.format(model_name))
    saver.restore(sess, tf.train.latest_checkpoint('./models/'))
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    y = graph.get_tensor_by_name("y:0")
    with_drop = graph.get_tensor_by_name('with_drop:0')
    normalization = graph.get_tensor_by_name('normalization:0')
    accuracy_operation = graph.get_tensor_by_name('accuracy_operation:0')
    
    X_test_norm = sess.run(normalization, feed_dict={x: X_test_norm})
    test_loss, test_accuracy = evaluate(X_test_norm, y_test)
    
    print("Test Accuracy = {:.3f}".format(test_accuracy))
    print("Test Loss = {:.3f}".format(test_loss))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
import cv2
import glob
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(20,15))
imgpaths_sample = glob.glob('./new_signs/*.jpg')
raw_imgs = []
new_img_labels = []
scaled_imgs = []
count = 1
for imgpath in imgpaths_sample:
    label = imgpath.split('/')[2].split('.')[0]
    new_img_labels.append(label)
    
    img_bgr = cv2.imread(imgpath)
    img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    gray_img = gray_clahe(img)
    scaled_img = cv2.resize(gray_img, (32,32))
    raw_imgs.append(img)
    scaled_imgs.append(scaled_img)
    
    # plot RGB original image
    plt.subplot(4,5, count)
    count += 1
    plt.title(all_labels[int(label),1], fontsize=18)
    plt.tight_layout()
    plt.imshow(img)
    
new_signs = np.array(scaled_imgs, dtype=np.float32).reshape(len(scaled_imgs), 32, 32, 1)

### Predict the Sign Type for Each Image

In [ ]:
with tf.Session() as sess:

    saver = tf.train.import_meta_graph('{}.meta'.format(model_name))
    saver.restore(sess, tf.train.latest_checkpoint('./models/'))
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    with_drop = graph.get_tensor_by_name('with_drop:0')
    normalization = graph.get_tensor_by_name('normalization:0')
    logits = graph.get_tensor_by_name('logits:0')
    
    new_signs_norm = sess.run(normalization, feed_dict={x: new_signs})
    
    predictions = sess.run(tf.argmax(logits, 1), feed_dict={x: new_signs_norm, with_drop:False})
    
    for indx in range(len(new_signs)):
        new_sign , new_sing_label = new_signs[indx], new_img_labels[indx]
        prediction = predictions[indx]
        print('Predict: {:<50} --- Ground True: {}'.format(all_labels[int(prediction),1], all_labels[int(new_sing_label),1]))

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('{}.meta'.format(model_name))
    saver.restore(sess, tf.train.latest_checkpoint('./models/'))

    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    y = graph.get_tensor_by_name("y:0")
    with_drop = graph.get_tensor_by_name('with_drop:0')
    normalization = graph.get_tensor_by_name('normalization:0')
    accuracy_operation = graph.get_tensor_by_name('accuracy_operation:0')
    
    new_signs_norm = sess.run(normalization, feed_dict={x: new_signs})
    num_examples = len(new_signs_norm)
    
    new_signs_accuracy = sess.run(accuracy_operation,feed_dict={x:new_signs_norm, y:new_img_labels, with_drop:False})
    
    print("Test Accuracy = {:.1f}%".format(new_signs_accuracy * 100))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
import matplotlib.image as mpimg

signs_list = []
for img_num in range(num_labels):
    img = mpimg.imread('./signs/{}.png'.format(img_num))
    signs_list.append(img)

In [ ]:
import matplotlib.image as mpimg
%matplotlib inline

def plot_sign(sign, values, index, nom):
    fig = plt.figure(nom, figsize=(15,4))
    
    ax0 = fig.add_subplot(1, 3, 1)
    ax0.imshow(sign)
    ax0.axis('off')
    
    x = np.arange(len(values))
    ax1 = fig.add_subplot(1, 3, 2)
    ax1.barh(x, values)
    y_labels = ['{:.1f}%'.format(x * 100) for x in values]
    ax1.set_yticks(x)
    ax1.set_yticklabels(y_labels, fontsize=13)
    
    ax2 = fig.add_subplot(5, 5, 4)
    ax2.imshow(signs_list[index[4]])
    ax2.axis('off')
    ax3 = fig.add_subplot(5, 5, 9)
    ax3.imshow(signs_list[index[3]])
    ax3.axis('off')
    ax4 = fig.add_subplot(5, 5, 14)
    ax4.imshow(signs_list[index[2]])
    ax4.axis('off')
    ax5 = fig.add_subplot(5, 5, 19)
    ax5.imshow(signs_list[index[1]])
    ax5.axis('off')
    ax6 = fig.add_subplot(5, 5, 24)
    ax6.imshow(signs_list[index[0]])
    ax6.axis('off')

In [ ]:
def plot_top_five(top_five):
    top_five_values = top_five.values
    top_five_indices = top_five.indices
    for indx in range(len(raw_imgs)):
        values = top_five_values[indx]
        indices = top_five_indices[indx]
        plot_sign(raw_imgs[indx], values, indices, indx)
        plt.show()
        print()

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('{}.meta'.format(model_name))
    saver.restore(sess, tf.train.latest_checkpoint('./models/'))
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    with_drop = graph.get_tensor_by_name('with_drop:0')
    normalization = graph.get_tensor_by_name('normalization:0')
    logits = graph.get_tensor_by_name('logits:0')
    
    new_signs_norm = sess.run(normalization, feed_dict={x: new_signs})
    top_five = sess.run(tf.nn.top_k(tf.nn.softmax(logits), k=5), feed_dict={x: new_signs_norm, with_drop: False})
    
    plot_top_five(top_five)

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input, with_drop: True})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(8, 6, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        plt.tight_layout()
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

In [ ]:
import tensorflow as tf
import random

# new_sign_sample = new_signs[random.randint(0, len(new_signs)-1)].reshape(1, 32, 32, 1)
new_sign_sample = new_signs[1].reshape(1, 32, 32, 1)
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('{}.meta'.format(model_name))
    saver.restore(sess, tf.train.latest_checkpoint('./models/'))
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    with_drop = graph.get_tensor_by_name('with_drop:0')
    normalization = graph.get_tensor_by_name('normalization:0')
    
    conv1 =  graph.get_tensor_by_name("conv1/conv:0")

    new_sign_norm = sess.run(normalization, feed_dict={x: new_sign_sample})
    outputFeatureMap(new_sign_norm, conv1)